In [2]:
require 'dbi'
require 'daru'
require 'nyaplot'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

false

In [3]:
db = DBI.connect("dbi:SQLite3:/home/dan/workspace/aact_import/aact")

terminal_statuses = ['Completed','No longer available','Suspended','Terminated','Withdrawn','Withheld']

["Completed", "No longer available", "Suspended", "Terminated", "Withdrawn", "Withheld"]

## Extract Data

In [4]:
studies = Daru::DataFrame.from_sql(db, 'select nct_id, overall_status, start_date, completion_date, completion_date_type  from clinical_study')

#<Daru::DataFrame:114468900 @name = cd8694b8-baf9-4660-90f1-449c0b3ecb44 @size = 211437>
           completion completion     nct_id overall_st start_date 
         0 March 2012     Actual NCT0000010 Terminated  July 2002 
         1                       NCT0000010 Active, no            
         2                       NCT0000010  Completed            
         3                       NCT0000010  Completed            
         4                       NCT0000010  Completed            
         5                       NCT0000010  Completed            
         6 December 1     Actual NCT0000011  Completed August 199 
         7 December 1     Actual NCT0000011  Completed October 19 
         8                       NCT0000011 Active, no March 1994 
         9                       NCT0000012  Completed January 19 
        10                       NCT0000013 Active, no October 19 
        11 August 200     Actual NCT0000013  Completed January 19 
        12  June 1994     Actual NCT0000011  Completed December 1 
        13                       NCT0000011  Completed            
        14                       NCT0000011 Active, no            
       ...        ...        ...        ...        ...        ...

In [5]:
# set blank start dates to nil
studies[:start_date] = studies[:start_date].map { |d| d == '' ? nil : d }

# convert non-nil start dates to Date
studies[:start_date] = studies[:start_date].map { |d| Date.strptime(d, '%b %Y') if !d.nil? }; nil

studies[:completion_date] = studies[:completion_date].map { |d| d == '' ? nil : d }
studies[:completion_date] = studies[:completion_date].map { |d| Date.strptime(d, '%b %Y') if !d.nil? }; nil

## Only include studies which started in or after 2006

In [9]:
recent_studies = studies.filter_rows { |s| !s[:start_date].nil? }
recent_studies = recent_studies.filter_rows { |s| (s[:start_date] > Date.parse('2006-01-01')) }
recent_studies = recent_studies.filter_rows { |s| terminal_statuses.include? s[:overall_status] }
recent_studies.size

90744

In [48]:
status_counts = recent_studies.group_by(:overall_status).count
status_counts[:status] = status_counts.index.to_a
status_counts.plot type: :bar, x: :status, y: :nct_id do |plot, diagram|
  plot.x_label ''
  plot.y_label ''
  plot.rotate_x_label -45
  plot.margin({top: 30, bottom: 120, left: 80, right: 30})
end

#<Nyaplot::Frame:0x00000016826408 @properties={:panes=>[#<Nyaplot::Plot:0x00000016823028 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000016826958 @properties={:type=>:bar, :options=>{:x=>:status, :y=>:nct_id}, :data=>"b46504b0-9b30-4be1-8e2a-d5b406827b4c"}, @xrange=["Completed", "No longer available", "Suspended", "Terminated", "Withdrawn", "Withheld"], @yrange=[0, 76860]>], :options=>{:x_label=>"", :y_label=>"", :rotate_x_label=>-45, :margin=>{:top=>30, :bottom=>120, :left=>80, :right=>30}, :width=>700, :xrange=>["Completed", "No longer available", "Suspended", "Terminated", "Withdrawn", "Withheld"], :yrange=>[0, 76860]}}>], :data=>{"b46504b0-9b30-4be1-8e2a-d5b406827b4c"=>#<Nyaplot::DataFrame:0x00000016826ed0 @name="b46504b0-9b30-4be1-8e2a-d5b406827b4c", @rows=[{:completion_date=>76860, :completion_date_type=>76860, :nct_id=>76860, :start_date=>76860, :status=>"Completed"}, {:completion_date=>19, :completion_date_type=>19, :nct_id=>19, :start_date=>19, :status=>"No longer available"}, {:completion_date=>852, :completion_date_type=>852, :nct_id=>852, :start_date=>852, :status=>"Suspended"}, {:completion_date=>8822, :completion_date_type=>8822, :nct_id=>8822, :start_date=>8822, :status=>"Terminated"}, {:completion_date=>3607, :completion_date_type=>3607, :nct_id=>3607, :start_date=>3607, :status=>"Withdrawn"}, {:completion_date=>584, :completion_date_type=>584, :nct_id=>584, :start_date=>584, :status=>"Withheld"}]>}, :extension=>[]}>

In [47]:
status_counts[:percent] = (status_counts.collect(:row) { |c| c[:nct_id].to_f.round(2) } / recent_studies.size).round(2)

status_counts.plot type: :bar, x: :status, y: :percent do |plot, diagram|
  plot.x_label ''
  plot.y_label ''
  plot.rotate_x_label -45
  plot.margin({top: 30, bottom: 120, left: 80, right: 30})
end

#<Nyaplot::Frame:0x00000019f65298 @properties={:panes=>[#<Nyaplot::Plot:0x00000019f62840 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000019f657e8 @properties={:type=>:bar, :options=>{:x=>:status, :y=>:percent}, :data=>"a8b1ae28-7a07-4d30-978a-495d2a7e7829"}, @xrange=["Completed", "No longer available", "Suspended", "Terminated", "Withdrawn", "Withheld"], @yrange=[0, 0.85]>], :options=>{:x_label=>"", :y_label=>"", :rotate_x_label=>-45, :margin=>{:top=>30, :bottom=>120, :left=>80, :right=>30}, :width=>700, :xrange=>["Completed", "No longer available", "Suspended", "Terminated", "Withdrawn", "Withheld"], :yrange=>[0, 0.85]}}>], :data=>{"a8b1ae28-7a07-4d30-978a-495d2a7e7829"=>#<Nyaplot::DataFrame:0x00000019f65d60 @name="a8b1ae28-7a07-4d30-978a-495d2a7e7829", @rows=[{:completion_date=>76860, :completion_date_type=>76860, :nct_id=>76860, :start_date=>76860, :status=>"Completed", :percent=>0.85}, {:completion_date=>19, :completion_date_type=>19, :nct_id=>19, :start_date=>19, :status=>"No longer available", :percent=>0.0}, {:completion_date=>852, :completion_date_type=>852, :nct_id=>852, :start_date=>852, :status=>"Suspended", :percent=>0.01}, {:completion_date=>8822, :completion_date_type=>8822, :nct_id=>8822, :start_date=>8822, :status=>"Terminated", :percent=>0.1}, {:completion_date=>3607, :completion_date_type=>3607, :nct_id=>3607, :start_date=>3607, :status=>"Withdrawn", :percent=>0.04}, {:completion_date=>584, :completion_date_type=>584, :nct_id=>584, :start_date=>584, :status=>"Withheld", :percent=>0.01}]>}, :extension=>[]}>